In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import os
os.chdir("..")

In [5]:
%%capture

%run examples/cosim_sil_example.py

AttributeError: module '__main__' has no attribute '__spec__'

Data received at /power: {'power': 2.14}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.44}
Data received at /power: {'power': 2.46}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.15}
Data received at /power: {'power': 2.11}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.33}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.64}
Data received at /power: {'power': 2.42}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.42}
Data received at /power: {'power': 2.09}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.25}
Data received at /power: {'power': 2.38}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.31}
Data received at /power: {'power': 2.22}
Data received at /power: {'power': 12.03}
Data received at /power: {'power': 2.46}
Data received at /power: {'power': 2.14}
Data rec

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\Users\kilian\anaconda3\envs\Synergy\Lib\site-packages\urllib3\connection.py", line 200, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kilian\anaconda3\envs\Synergy\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\kilian\anaconda3\envs\Synergy\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] Ein Verbindungsversuch ist fehlgeschlagen, da die Gegenstelle nach einer bestimmten Zeitspanne nicht richtig reagiert hat, oder die hergestellte Verbindung war fehlerhaft, da der verbundene Host nicht reagiert hat

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\kilian\anaconda3\envs\Synergy\Lib\site-packages\urllib3\connectionpool.py", line 790, in u

In [ ]:
# Get data of baseline simulation
with open ("data.csv", "r") as f:
    baseline_experiment = pd.read_csv(f, index_col=0)
baseline_experiment.index = pd.to_datetime(baseline_experiment.index)

In [ ]:
%%capture

%run examples/cosim_sil_cacu_example.py

In [ ]:
with open ("data.csv", "r") as f:
    carbon_aware_experiment = pd.read_csv(f, index_col=0)
carbon_aware_experiment.index = pd.to_datetime(carbon_aware_experiment.index)

In [ ]:
RED = "#D65D62"
GREEN = "#79ae7c"

def plot_evaluation(measurements_df, axs, title: str, left: bool):
    axs[0].set_title(title)

    # Unpack data
    p_prod = measurements_df["p_solar"]
    p_computing_system = - measurements_df["p_computing_system"]
    p_grid = - measurements_df["p_grid"]
    battery_soc = measurements_df["battery_soc"]
    battery_min_soc = measurements_df["battery_min_soc"]
    carbon = measurements_df["carbon_intensity"]

    # Plot Production and Consumption
    p_prod.plot(ax=axs[0], color="#6b9a6e", label="production")
    p_computing_system.plot(ax=axs[0], color="#ce3d43", label="consumption", linestyle=":")
    axs[0].set_ylabel("power (W)")
    axs[0].legend(frameon=False)

    # Plot Production/Consumption delta
    x = p_prod - p_computing_system
    x.plot(ax=axs[1], alpha=0)
    axs[1].fill_between(x.index, 0, x.values, where=x.values>0, color=GREEN)
    axs[1].fill_between(x.index, 0, x.values, where=x.values<0, color=RED)
    axs[1].set_ylabel("power\ndelta (W)")

    # Plot Battery State of Charge
    battery_soc.plot(ax=axs[2], alpha=0)
    axs[2].fill_between(battery_soc.index, 0, battery_soc.values * 100, color=GREEN)
    axs[2].set_ylim(0, 100)
    axs[2].set_ylabel("battery state\nof charge (%)")
    (battery_min_soc * 100).plot(ax=axs[2], linestyle="--", linewidth=.8, color="black", label="max discharge")
    h, l = axs[2].get_legend_handles_labels()
    axs[2].legend(h[1:], l[1:], frameon=False)

    # Plot grid power and carbon emission
    p_grid[p_grid < 0] = 0
    p_grid.plot(ax=axs[3], color=RED, linestyle="-")
    axs[3].fill_between(p_grid.index, 0, p_grid.values.astype(float), color=RED)
    axs[3].set_ylim(0, 30)
    axs[3].set_yticks([0, 20])
    axs[3].set_ylabel("grid power\nusage (W)")
    ax3_twin = axs[3].twinx()
    carbon.plot(ax=ax3_twin, color="#333", linewidth=.8)
    ax3_twin.set_ylim(0, 350)
    if left:
        ax3_twin.set_yticks([])
        label_x = 0.35
    else:
        ax3_twin.set_ylabel("carbon intensity\n(gCO2/kWh)")
        ax3_twin.set_yticks([0, 100, 200, 300])
        label_x = 0.45
    h1, l1 = axs[3].get_legend_handles_labels()
    h2, l2 = ax3_twin.get_legend_handles_labels()
    axs[3].legend([h1[0], h2[0]], ["grid power usage", "carbon emission"], frameon=False, bbox_to_anchor=(label_x, 0.63))

    # Plot accumulated carbon emission
    emissions = p_grid * carbon  # 60 Ws * gCO2/kWh  //  gCO2/60000 // kgCO2/60000/1000
    emissions /= 60000
    emissions.cumsum().plot(ax=axs[4], color="#555", linewidth=.8)
    axs[4].fill_between(emissions.index, emissions.cumsum().values, color="#ddd")
    axs[4].set_ylabel("accumulated\nemissions (gCO2)")
    axs[4].set_ylim(0, 43)

fig, axs = plt.subplots(ncols=2, nrows=5, sharex="col", sharey="row", figsize=(10,5.5))
fig.tight_layout(pad=0)
plot_evaluation(baseline_experiment, axs[:,0], title="Baseline experiment", left=True)
plot_evaluation(carbon_aware_experiment, axs[:,1], title="Carbon-aware experiment", left=False)
fig.align_ylabels()